In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [58]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [45]:
missing_values_count = train_data.isnull().sum()
missing_values_count[0:12]

In [46]:
missing_values_count = test_data.isnull().sum()
missing_values_count[0:11]

In [52]:
train_data = train_data.dropna(axis = 0, subset = ["Embarked"])#odstranit řádek (axis=0) kde chybí hodnota ve sloupci "Embarked" - jsou jenom 2
train_data['Sex'] = train_data['Sex'].replace('female', 0 )
train_data['Sex'] = train_data['Sex'].replace('male', 1)
train_data['Age'] = train_data['Age'].fillna(train_data.Age.mean())#chybějící hodnoty "Age" se vyplní průměrem "Age"
#train_data.Age = train_data.Age.fillna(0)
#train_data = train_data.dropna(axis = 0, subset = ["Age"])
train_data['Embarked_s'] = train_data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
train_data['Embarked_c'] = train_data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
train_data['Embarked_q'] = train_data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

In [53]:
train_data.head()

In [70]:
test_data['Fare'] = test_data['Fare'].fillna(test_data.Age.mean())
test_data['Age'] = test_data['Age'].fillna(test_data.Age.mean())
#test_data['Age'] = test_data['Age'].fillna(0)
test_data['Sex'] = test_data['Sex'].replace('female', 0)
test_data['Sex'] = test_data['Sex'].replace('male', 1)
test_data['Embarked_s'] = test_data['Embarked'].replace('Q',0).replace('C',0).replace('S',1)
test_data['Embarked_c'] = test_data['Embarked'].replace('S',0).replace('Q',0).replace('C',1)
test_data['Embarked_q'] = test_data['Embarked'].replace('C',0).replace('S',0).replace('Q',1)

In [71]:
test_data.head()

In [56]:
train_data.isnull().sum()

In [72]:
test_data.isnull().sum()

In [62]:
y = train_data.Survived

features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked_s", "Embarked_q", "Embarked_c"]
X = train_data[features]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

#model = RandomForestClassifier(n_estimators=100, random_state=1)
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(train_X, train_y)
val_predictions = model.predict(val_X)

#mean_acc = model.score(train_X, train_y)

In [63]:
(val_predictions == val_y).sum().astype(float)/val_y.shape[0]

In [64]:
confusion_matrix(val_y, val_predictions)

In [65]:
print(classification_report(val_y, val_predictions))

In [73]:
#trénování na všech trénovacích datech
#model_on_full_data = RandomForestClassifier(n_estimators=100, random_state=1)
model_on_full_data = LogisticRegression(solver='liblinear', random_state = 1)

model_on_full_data.fit(X, y)

In [75]:
#predikce na předpřipravených testovacích datech
test_X = test_data[features]
 
test_preds = model_on_full_data.predict(test_X)

In [76]:
print(test_preds)

In [77]:
output = pd.DataFrame({'PassengerId': test_data['PassengerId'],'Survived': test_preds})
output.to_csv('submission.csv', index=False)

In [78]:
print(output)